In [1]:
import json, argparse, torch, sys, random, gc, os
import numpy as np
import pandas as pd
import functools
from itertools import chain
from functools import partial
from pathlib import Path

# Transformer 
from transformers import (AutoTokenizer, Trainer, TrainingArguments,
                          AutoModelForTokenClassification, DataCollatorForTokenClassification,
                          DebertaV2Config, DebertaV2ForTokenClassification)
from datasets import Dataset, features
from typing import Iterable, Any, Callable
from sklearn.model_selection import train_test_split

/home/pytorch/Ritesh-D11AD-05/Ritesh_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MAX_LENGTH = 1024
model_path = "models/deberta3small_address_model_1024"
threshold = 0.99

In [3]:
def tokenize(example, tokenizer):
    tokens = []
    token_map = []
    idx = 0
    for t, ws in zip(example["tokens"], example["trailing_whitespace"]):
        tokens.append(t)
        token_map.extend([idx]*len(t))
        if ws:
            tokens.append(" ")
            token_map.append(-1) 
        idx += 1
    # Does not truncate the text and concate all words together 
    # Do not need to have extra space as we have already include it in the previous tokenization
    tokenized = tokenizer("".join(tokens), return_offsets_mapping=True, truncation=False)
    return {**tokenized, "token_map": token_map}

In [4]:
from scipy.special import softmax
from datasets import Dataset 
# Model Inferer
class ModelInfer:
    def __init__(self, all_labels, id2label):
        self.all_labels = all_labels
        # self.label2id = label2id
        self.id2label = id2label
        self.model_path = model_path
        self.max_length = 1024
        self.infer_dir = "/kaggle/working/infer" # Model infer output 
        self.num_proc = 3 # 3 processors
        self.threshold = threshold # Threashold
        self.load_model()
        
    def load_model(self):
        # Create the tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path) 
        # Create the model
        self.model = AutoModelForTokenClassification.from_pretrained(self.model_path)        
#         # Load the fine-tuned adapter layer on top of base model
#         self.model = self.model.to(DEVICE)
        print(f"Complete loading pretrained LLM model")
    
    def post_processing_preds(self, preds):
        preds_final = []
        preds_softmax = np.exp(preds) / np.sum(np.exp(preds), axis=2).reshape(preds.shape[0],
                                                                              preds.shape[1],
                                                                              1)
        # Get the maximal value as the final preds
        preds = preds.argmax(-1)
        preds_without_O = preds_softmax[:,:,:2].argmax(-1) # Prob of entity labels (like 'NAME_STUDENT')
        O_preds = preds_softmax[:,:,2] # Prob for 'O'
        print()
        # If preds for 'O' > 0.99, select preds of 'O'. Otherwise, select preds for entity labels.  
        preds_final = np.where(O_preds < self.threshold, preds_without_O, preds)
        return preds_final        
        
    def infer_preds(self, ds):
        # Tokenize the dataset using customized Tokenizer (the same as Training Tokenizer)
        tokenized_ds = ds.map(tokenize, fn_kwargs={"tokenizer": self.tokenizer}, num_proc=2)
        # Create data loader
        data_collator = DataCollatorForTokenClassification(self.tokenizer,
                                                           pad_to_multiple_of=16)
        # Arguments (infer only)
        args = TrainingArguments(output_dir=self.infer_dir,
                                 per_device_eval_batch_size=1, 
                                 report_to="none")
        # Create the trainer 
        trainer = Trainer(model=self.model, 
                          args=args, 
                          data_collator=data_collator, 
                          tokenizer=self.tokenizer)
        
        # predict for that split
        preds = trainer.predict(tokenized_ds).predictions
                
        # Clear the unused memory
        del self.model, data_collator, trainer, args 
        clear_memory()
        preds_final = self.post_processing_preds(preds)
        return preds_final, tokenized_ds

In [5]:
all_labels = ['B-STREET_ADDRESS', 'I-STREET_ADDRESS', 'O']

In [6]:
import ctypes
libc = ctypes.CDLL("libc.so.6")
def clear_memory():
    libc.malloc_trim(0)
    torch.cuda.empty_cache()
    gc.collect()

In [7]:
clear_memory()

In [8]:
test_data = pd.read_csv("custom_test_df.csv")
# test_data = test_data.sample(n=500)
test_ds = Dataset.from_dict({
    "full_text": test_data["full_text"].tolist(),
    "document": test_data["document"].tolist(),
    "tokens": test_data["tokens"].tolist(),
    "trailing_whitespace": test_data["trailing_whitespace"].tolist(),
})
print(f"Total number of test dataset {len(test_ds)}")
config = json.load(open(Path(model_path) / "config.json"))
id2label = config["id2label"]
# Load the pretrained model and make the predictions
inferer = ModelInfer(all_labels, id2label)
preds_final, tokenized_ds = inferer.infer_preds(test_ds) 

Total number of test dataset 157
Complete loading pretrained LLM model


Map (num_proc=2): 100%|████████████████████████████████████████████████████████| 157/157 [00:01<00:00, 87.94 examples/s]
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.49 GiB. GPU 0 has a total capacity of 14.61 GiB of which 105.56 MiB is free. Including non-PyTorch memory, this process has 14.50 GiB memory in use. Of the allocated memory 12.23 GiB is allocated by PyTorch, and 1.58 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
preds_final.shape

In [ ]:
preds_final[0,:-10]

In [ ]:
results = []
for p, token_map, offsets, tokens, doc in zip(preds_final,
                                              tokenized_ds["token_map"], 
                                              tokenized_ds["offset_mapping"],
                                              tokenized_ds["tokens"],
                                              tokenized_ds["document"]):
    for token_pred, (start_idx, end_idx) in zip(p, offsets):
        try:
            label_pred = id2label[str(token_pred)]
            if start_idx + end_idx == 0: 
                continue

            if token_map[start_idx] == -1:
                start_idx += 1

            # ignore "\n\n"
            while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace():
                start_idx += 1

            if start_idx >= len(token_map): 
                break

            token_id = token_map[start_idx]

            # ignore "O" predictions and whitespace preds
            if label_pred != "O" and token_id != -1:
                results.append({
                        "document": doc,
                        "token": token_id,
                        "label": label_pred,
                        "token_str": tokens[token_id]
                    })
        except Exception as e:
            print(f"Error {e}")
            print(f"token_map {len(token_map)} and {token_pred}  {start_idx} {end_idx}")
            sys.exit(-1)

In [ ]:
results

In [ ]:
def remove_duplicates(df):
    # Sort by the document and token
    df.sort_values(by=['document', 'token'])
    # Combine three columns 
    df['triplet'] = df[["document", "token", "label"]].apply(lambda row: '_'.join(row.values.astype(str)), axis=1) 
    # display(df)
    # Drop duplicated triplets and keep the first one as unique row
    df = df.drop_duplicates(subset=["triplet"], keep='first')
    # Regenerate 'row_id'
    df['row_id'] = list(range(len(df)))    
    df = df.reset_index(drop=True, inplace=False) 
    print("Remove duplicates")
#     display(df)
    return df

In [ ]:
test_df = pd.DataFrame(results)
test_df = remove_duplicates(test_df)
test_df = test_df[["row_id", "document", "token", "label"]]

In [ ]:
test_df

In [ ]:
test_data[test_data['document']==4744].full_text

In [ ]:
test_data[test_data['document']==4744].tokens.tolist()[0].split(',')